In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [105]:
def find_split_point(iteration):
    split_point = []
    for i in range(len(iteration)):
        if iteration[i] == 2000:
            split_point.append(i)
    return split_point

In [243]:
def parse_log_file(path):
    train_loss = []
    iteration = []
    epoch_info = []
    with open(path) as file:
        l = file.readline()
        while len(l)>0:
            l = file.readline()
            if l.startswith('Iter'):
                start = l.index('Iter')
                end = l.index(',')
                iteration.append(int(l[start+4:end].strip()))
                train_loss.append(float(l[l.index('Train Loss:')+11:].strip()))
            if l.startswith('Epoch'):
                epoch_info.append(l)
    num_epoch = len(epoch_info)
    split_points = find_split_point(iteration)
    iteration_by_epoch = []
    for i  in range(len(split_point)-1):
        split = split_point[i]
        next_split = split_point[i+1]
        iteration_by_epoch.append(iteration[split:next_split])
    iteration_by_epoch.append(iteration[next_split:])
    total_iteration_per_epoch = []
    for i in range(num_epoch):
        total_iteration_per_epoch.append(iteration_by_epoch[i][-1])
    total_iteration_per_epoch = np.array(total_iteration_per_epoch)
    print(total_iteration_per_epoch)
    print(num_epoch)
    N = len(iteration)
    loss_train_epoch = []
    loss_test_epoch = []
    f1_test_epoch = []
    acc_test_epoch = []
    time_epoch = []
    for i in range(num_epoch):
        epoch = epoch_info[i]
        train_start = epoch.index('train loss')
        train_end = epoch.index('test loss')
        test_start = train_end
        test_end = epoch.index('test acc')
        acc_start = test_end
        acc_end = epoch.index("test f1")
        f1_start = acc_end
        time_start = epoch.index('epoch time')
        loss_train_epoch.append(float(epoch[train_start+11:train_end-2].strip()))
        loss_test_epoch.append(float(epoch[test_start+10:test_end-2].strip()))
        acc_test_epoch.append(float(epoch[acc_start+10:acc_end-3]))
        f1_test_epoch.append(float(epoch[f1_start+9:time_start-2].strip()))
        time_epoch.append(float(epoch[time_start+12:].strip()))
    average_time_by_epoch = np.array(time_epoch)/total_iteration_per_epoch.reshape(-1,num_epoch)
    summary_epoch = pd.DataFrame({'time':time_epoch,
                'test f1':f1_test_epoch,'train loss': loss_train_epoch,
                'test loss':loss_test_epoch,
                'time per iter':average_time_by_epoch.flatten()}) 
    #create per iteration summary 
    end_points = split_point[1:]
    end_points.append(len(iteration))
    duplicate_num = np.array(end_points)-np.array(split_point)

    time =[]
    cut = len(time)# this is to get rid of unfinished epoch
    for i in range(num_epoch):
        time +=[summary_epoch['time per iter'].values[i]]*duplicate_num[i]
    accumulated_time = [np.sum(time[:i]) for i in range(1,N+1)]
    iteration_summary = pd.DataFrame({'iteration':2000*np.arange(1,cut+1),
                                     'train loss':train_loss[:cut],
                                     'time per iter':time[:cut],
                                     'accumulated time':accumulated_time[:cut]})
    return iteration_summary,summary_epoch,train_loss[-1],

In [237]:
iteration_summary,summary_epoch,train_loss=parse_log_file(path)

[148000 148000 148000 148000 148000 148000 148000 148000 148000 148000]
10


In [238]:
summary_epoch

,test f1,test loss,time,time per iter,train loss
0,0.85,0.134788,3839.790822,0.025945,0.150812
1,0.86,0.131340,3542.279176,0.023934,0.133209
2,0.86,0.128890,3572.740484,0.024140,0.129927
3,0.86,0.127537,3570.939420,0.024128,0.127932
4,0.86,0.126627,3605.136089,0.024359,0.126526
5,0.86,0.125799,3534.149846,0.023879,0.125456
6,0.86,0.125335,3564.928952,0.024087,0.124608
7,0.86,0.125117,3564.640195,0.024085,0.123885
8,0.86,0.124530,3544.977293,0.023953,0.123253
9,0.86,0.124762,3573.054511,0.024142,0.122712


In [241]:
iteration_summary,summary_epoch,train_loss=parse_log_file('Log/log_single_prof.out')

[148000 148000 148000 148000 148000 148000 148000 148000 148000 148000]
10


In [242]:
summary_epoch

,test f1,test loss,time,time per iter,train loss
0,0.85,0.135644,3904.062772,0.026379,0.175167
1,0.85,0.130832,3623.557158,0.024483,0.133676
2,0.85,0.128494,3629.061251,0.024521,0.130029
3,0.85,0.127210,3637.042340,0.024575,0.127945
4,0.85,0.126071,3654.882662,0.024695,0.126475
5,0.86,0.126311,3622.479209,0.024476,0.125335
6,0.86,0.126141,3635.214981,0.024562,0.124404
7,0.85,0.124614,3647.205502,0.024643,0.123627
8,0.85,0.125877,3622.570585,0.024477,0.122950
9,0.85,0.125913,3631.198354,0.024535,0.122376
